In [1]:
import sqlite3
import pandas as pd

In [2]:
%load_ext sql

To illustrate how join operations work, we first create a silly database that will help with examples.

In [3]:
conn = sqlite3.connect("foo.db")
cur = conn.cursor()

In [4]:
%sql sqlite:///foo.db

Connecting to 'sqlite:///foo.db'

In [14]:
%config SqlMagic.displaylimit = 5

In [16]:
%%sql

create table profs (
    ID INT,
    f_name VARCHAR(50),
    l_name VARCHAR(50)
);

insert into profs (ID, f_name, l_name)
VALUES
(1, 'Jason', 'Smith'),
(2, 'Randall', 'Paul'),
(3, 'Terrie', 'Torres'),
(4, 'Ken', 'Davis'),
(5, 'Tiernan', 'Fogarty'),
(6, 'Jim', 'Fischer'),
(7, 'Dr.', 'Deb')

Running query in 'sqlite:///foo.db'

7 rows affected.

++
||
++
++

In [20]:
%%sql

create table subject (
    ID INT,
    subject VARCHAR(50)
);

insert into subject (ID, subject)
VALUES
(0, 'Comm'),
(2, 'Trig'),
(4, 'Discrete'),
(5, 'Calc'),
(7, 'Algebra'),
(8, 'English'),
(9, 'CS')

Running query in 'sqlite:///foo.db'

7 rows affected.

++
||
++
++

In [6]:
%sql select * from profs;

Running query in 'sqlite:///foo.db'

ID,f_name,l_name
1,Jason,Smith
2,Randall,Paul
3,Terrie,Torres
4,Ken,Davis
5,Tiernan,Fogarty
6,Jim,Fischer
7,Dr.,Deb


In [8]:
%sql select * from subject;

Running query in 'sqlite:///foo.db'

ID,subject
0,Comm
2,Trig
4,Discrete
5,Calc
7,Algebra
8,English
9,CS


### Join Operations

*Join Operations* are used to take relevant pieces of two or more tables in a database and combine them into one table.  In order to avoid a *Cartesian Join* like we saw in the last notebook, we have to specify a key that tells SQL how the tables are connected.

There are two types of join operations that we will study in this class:
+ Inner Join
+ Left Outer Join
+ Right Outer Join
+ Full Outer Join

#### Inner Joins

An *inner join* joins together only the rows for which the key values exist in both tables.

##### Example 1

We are going to select f_name, l_name, and subject from both the profs and subject tables and put them into one table.  An inner join only displays the rows that have a key value in the intersection of the key values for both tables.  

In [21]:
%sql select * from profs;

Running query in 'sqlite:///foo.db'

ID,f_name,l_name
1,Jason,Smith
2,Randall,Paul
3,Terrie,Torres
4,Ken,Davis
5,Tiernan,Fogarty
6,Jim,Fischer
7,Dr.,Deb


In [22]:
%sql select * from subject;

Running query in 'sqlite:///foo.db'

ID,subject
0,Comm
2,Trig
4,Discrete
5,Calc
7,Algebra
8,English
9,CS


In [25]:
%sql select P.ID, P.f_name, P.l_name, S.subject from profs P inner join subject S on P.ID = S.ID;

Running query in 'sqlite:///foo.db'

ID,f_name,l_name,subject
2,Randall,Paul,Trig
4,Ken,Davis,Discrete
5,Tiernan,Fogarty,Calc
7,Dr.,Deb,Algebra


Note that the intersection of profs.ID = $\{1,2,3,4,5,6,7 \}$ and subject.ID = $\{0,2,4,5,7,8,9\}$ is profs.ID$\bigcap$subject.ID $= \{2,4,5,7\}$.

$\Box$

#### Left Outer Join

Suppose that we are going to perform a left outer join on two tables $T_1$ and $T_2$ with common key column labeled ID.  Then, the rows displayed will be those whose keys are in the set $(T_1.ID-T_2.ID)\bigcup(T_1.ID\bigcap T_2.ID)$.

##### Example 2

In [27]:
%sql select P.ID, P.f_name, P.l_name, S.subject from profs P left outer join subject S on P.ID = S.ID;

Running query in 'sqlite:///foo.db'

ID,f_name,l_name,subject
1,Jason,Smith,None
2,Randall,Paul,Trig
3,Terrie,Torres,None
4,Ken,Davis,Discrete
5,Tiernan,Fogarty,Calc
6,Jim,Fischer,None
7,Dr.,Deb,Algebra


Again, the intersection of profs.ID = $\{1,2,3,4,5,6,7 \}$ and subject.ID = $\{0,2,4,5,7,8,9\}$ is profs.ID$\bigcap$subject.ID $= \{2,4,5,7\}$.

Also, profs.ID-subject.ID = $\{1,3,6\}$.

So, the rows shown are those rows with ID in (profs.ID-subject.ID)$\bigcup$(profs.ID$\bigcap$subject.ID) = $\{1,3,6\}\bigcup\{2,4,5,7\}=\{1,2,3,4,5,6,7\}$.

$\Box$

#### Right Outer Join

Suppose that we are going to perform a right outer join on two tables $T_1$ and $T_2$ with common key column labeled ID.  Then, the rows displayed will be those whose keys are in the set $(T_2.ID-T_1.ID)\bigcup(T_1.ID\bigcap T_2.ID)$.

##### Example 3

Unfortunately, sqlite3 does not have an actual right outer join operation.  However, with some finagling I was able to get the result that a right outer join would produce.

In [23]:
%sql select S.ID, P.f_name, P.l_name, S.subject from subject S left outer join profs P on P.ID = S.ID;

Running query in 'sqlite:///foo.db'

ID,f_name,l_name,subject
0,None,None,Comm
2,Randall,Paul,Trig
4,Ken,Davis,Discrete
5,Tiernan,Fogarty,Calc
7,Dr.,Deb,Algebra
8,None,None,English
9,None,None,CS


Again, the intersection of profs.ID = $\{1,2,3,4,5,6,7 \}$ and subject.ID = $\{0,2,4,5,7,8,9\}$ is profs.ID$\bigcap$subject.ID $= \{2,4,5,7\}$.

Also, subject.ID-profs.ID = $\{0,8,9\}$.

So, the rows shown are those rows with ID in (subject.ID-proffs.ID)$\bigcup$(profs.ID$\bigcap$subject.ID) = $\{0,8, 9\}\bigcup\{2,4,5,7\}=\{0,2,4,5,7,8,9\}$.

$\Box$

#### Full Outer Join

Suppose that we are going to perform a full outer join on two tables $T_1$ and $T_2$ with common key column labeled ID.  Then, the rows displayed will be those whose keys are in the set $T_1.ID\bigcup T_2.ID$.

##### Example 4

Unfortunately, sqlite3 does not have an actual full outer join operation.  However, using the UNION operator I was able to get the result that a full outer join would produce.

In [25]:
%%sql

select P.ID, P.f_name, P.l_name, S.subject from profs P left outer join subject S on P.ID = S.ID
UNION
select S.ID, P.f_name, P.l_name, S.subject from subject S left outer join profs P on P.ID = S.ID;

Running query in 'sqlite:///foo.db'

ID,f_name,l_name,subject
0,None,None,Comm
1,Jason,Smith,None
2,Randall,Paul,Trig
3,Terrie,Torres,None
4,Ken,Davis,Discrete
5,Tiernan,Fogarty,Calc
6,Jim,Fischer,None
7,Dr.,Deb,Algebra
8,None,None,English
9,None,None,CS


$\Box$

Now, we are going to look at some more interesting examples using the Chicago datasets.  To this end, we create a database that contains each of the three Chicago datasets that we have been using so far in this class.

In [4]:
connection = sqlite3.connect("chicago.db")
cursor = connection.cursor()

In [5]:
%sql sqlite:///chicago.db

Connecting to 'sqlite:///chicago.db'

In [6]:
schools = pd.read_csv('ChicagoPublicSchools.csv')

In [7]:
schools.to_sql('schools', connection, if_exists='replace', index=False,method="multi")

566

In [8]:
census = pd.read_csv('ChicagoCensusData.csv')

In [9]:
census.to_sql('census', connection, if_exists='replace', index=False,method="multi")

78

In [10]:
crime = pd.read_csv('ChicagoCrimeData.csv')

In [11]:
crime.to_sql('crime', connection, if_exists='replace', index=False,method="multi")

533

In [12]:
%config SqlMagic.displaylimit = 5

In [13]:
%sql select * from schools;

Running query in 'sqlite:///chicago.db'

School_ID,NAME_OF_SCHOOL,"Elementary, Middle, or High School",Street_Address,City,State,ZIP_Code,Phone_Number,Link,Network_Manager,Collaborative_Name,Adequate_Yearly_Progress_Made_,Track_Schedule,CPS_Performance_Policy_Status,CPS_Performance_Policy_Level,HEALTHY_SCHOOL_CERTIFIED,Safety_Icon,SAFETY_SCORE,Family_Involvement_Icon,Family_Involvement_Score,Environment_Icon,Environment_Score,Instruction_Icon,Instruction_Score,Leaders_Icon,Leaders_Score,Teachers_Icon,Teachers_Score,Parent_Engagement_Icon,Parent_Engagement_Score,Parent_Environment_Icon,Parent_Environment_Score,AVERAGE_STUDENT_ATTENDANCE,Rate_of_Misconducts__per_100_students_,Average_Teacher_Attendance,Individualized_Education_Program_Compliance_Rate,Pk_2_Literacy__,Pk_2_Math__,Gr3_5_Grade_Level_Math__,Gr3_5_Grade_Level_Read__,Gr3_5_Keep_Pace_Read__,Gr3_5_Keep_Pace_Math__,Gr6_8_Grade_Level_Math__,Gr6_8_Grade_Level_Read__,Gr6_8_Keep_Pace_Math_,Gr6_8_Keep_Pace_Read__,Gr_8_Explore_Math__,Gr_8_Explore_Read__,ISAT_Exceeding_Math__,ISAT_Exceeding_Reading__,ISAT_Value_Add_Math,ISAT_Value_Add_Read,ISAT_Value_Add_Color_Math,ISAT_Value_Add_Color_Read,Students_Taking__Algebra__,Students_Passing__Algebra__,9th Grade EXPLORE (2009),9th Grade EXPLORE (2010),10th Grade PLAN (2009),10th Grade PLAN (2010),Net_Change_EXPLORE_and_PLAN,11th Grade Average ACT (2011),Net_Change_PLAN_and_ACT,College_Eligibility__,Graduation_Rate__,College_Enrollment_Rate__,COLLEGE_ENROLLMENT,General_Services_Route,Freshman_on_Track_Rate__,X_COORDINATE,Y_COORDINATE,Latitude,Longitude,COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME,Ward,Police_District,Location
610038,Abraham Lincoln Elementary School,ES,615 W Kemper Pl,Chicago,IL,60614,(773) 534-5720,http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610038.pdf,Fullerton Elementary Network,NORTH-NORTHWEST SIDE COLLABORATIVE,No,Standard,Not on Probation,Level 1,Yes,Very Strong,99.0,Very Strong,99,Strong,74.0,Strong,66.0,Weak,65,Strong,70,Strong,56,Average,47,96.00%,2.0,96.40%,95.80%,80.1,43.3,89.6,84.9,60.7,62.6,81.9,85.2,52,62.4,66.3,77.9,69.7,64.4,0.2,0.9,Yellow,Green,67.1,54.5,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,813,33,NDA,1171699.458,1915829.428,41.92449696,-87.64452163,7,LINCOLN PARK,43,18,"(41.92449696, -87.64452163)"
610281,Adam Clayton Powell Paideia Community Academy Elementary School,ES,7511 S South Shore Dr,Chicago,IL,60649,(773) 535-6650,http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610281.pdf,Skyway Elementary Network,SOUTH SIDE COLLABORATIVE,No,Track_E,Not on Probation,Level 1,No,Average,54.0,Strong,66,Strong,74.0,Very Strong,84.0,Weak,63,Strong,76,Weak,46,Average,50,95.60%,15.7,95.30%,100.00%,62.4,51.7,21.9,15.1,29,42.8,38.5,27.4,44.8,42.7,14.1,34.4,16.8,16.5,0.7,1.4,Green,Green,17.2,27.3,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,521,46,NDA,1196129.985,1856209.466,41.76032435,-87.55673627,43,SOUTH SHORE,7,4,"(41.76032435, -87.55673627)"
610185,Adlai E Stevenson Elementary School,ES,8010 S Kostner Ave,Chicago,IL,60652,(773) 535-2280,http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610185.pdf,Midway Elementary Network,SOUTHWEST SIDE COLLABORATIVE,No,Standard,Not on Probation,Level 2,No,Strong,61.0,NDA,NDA,Average,50.0,Weak,36.0,Weak,NDA,NDA,NDA,Average,47,Weak,41,95.70%,2.3,94.70%,98.30%,53.7,26.6,38.3,34.7,43.7,57.3,48.8,39.2,46.8,44,7.5,21.9,18.3,15.5,-0.9,-1.0,Red,Red,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,1324,44,NDA,1148427.165,1851012.215,41.74711093,-87.73170248,70,ASHBURN,13,8,"(41.74711093, -87.73170248)"
609993,Agustin Lara Elementary Academy,ES,4619 S Wolcott Ave,Chicago,IL,60609,(773) 535-4389,http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609993.pdf,Pershing Elementary Network,SOUTHWEST SIDE COLLABORATIVE,No,Track_E,Not on Probation,Level 1,No,Average,56.0,Average,44,Average,45.0,Weak,37.0,Weak,65,Average,48,Average,53,Strong,58,95.50%,10.4,95.80%,100.00%,76.9,NDA,26,24.7,61.8,49.7,39.2,27.2,69.7,60.6,9.1,18.2,11.1,9.6,0.9,2.4,Green,Green,42.9,25,NDA,N

In [14]:
%sql select * from census;

Running query in 'sqlite:///chicago.db'

COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME,PERCENT_OF_HOUSING_CROWDED,PERCENT_HOUSEHOLDS_BELOW_POVERTY,PERCENT_AGED_16__UNEMPLOYED,PERCENT_AGED_25__WITHOUT_HIGH_SCHOOL_DIPLOMA,PERCENT_AGED_UNDER_18_OR_OVER_64,PER_CAPITA_INCOME,HARDSHIP_INDEX
1.0,Rogers Park,7.7,23.6,8.7,18.2,27.5,23939,39.0
2.0,West Ridge,7.8,17.2,8.8,20.8,38.5,23040,46.0
3.0,Uptown,3.8,24.0,8.9,11.8,22.2,35787,20.0
4.0,Lincoln Square,3.4,10.9,8.2,13.4,25.5,37524,17.0
5.0,North Center,0.3,7.5,5.2,4.5,26.2,57123,6.0


In [15]:
%sql select * from crime;

Running query in 'sqlite:///chicago.db'

ID,CASE_NUMBER,DATE,BLOCK,IUCR,PRIMARY_TYPE,DESCRIPTION,LOCATION_DESCRIPTION,ARREST,DOMESTIC,BEAT,DISTRICT,WARD,COMMUNITY_AREA_NUMBER,FBICODE,X_COORDINATE,Y_COORDINATE,YEAR,LATITUDE,LONGITUDE,LOCATION
3512276,HK587712,2004-08-28,047XX S KEDZIE AVE,890,THEFT,FROM BUILDING,SMALL RETAIL STORE,0,0,911,9,14.0,58.0,6,1155838.0,1873050.0,2004,41.8074405,-87.70395585,"(41.8074405, -87.703955849)"
3406613,HK456306,2004-06-26,009XX N CENTRAL PARK AVE,820,THEFT,$500 AND UNDER,OTHER,0,0,1112,11,27.0,23.0,6,1152206.0,1906127.0,2004,41.89827996,-87.71640551,"(41.898279962, -87.716405505)"
8002131,HT233595,2011-04-04,043XX S WABASH AVE,820,THEFT,$500 AND UNDER,NURSING HOME/RETIREMENT HOME,0,0,221,2,3.0,38.0,6,1177436.0,1876313.0,2011,41.81593313,-87.62464213,"(41.815933131, -87.624642127)"
7903289,HT133522,2010-12-30,083XX S KINGSTON AVE,840,THEFT,FINANCIAL ID THEFT: OVER $300,RESIDENCE,0,0,423,4,7.0,46.0,6,1194622.0,1850125.0,2010,41.74366532,-87.56246276,"(41.743665322, -87.562462756)"
10402076,HZ138551,2016-02-02,033XX W 66TH ST,820,THEFT,$500 AND UNDER,ALLEY,0,0,831,8,15.0,66.0,6,1155240.0,1860661.0,2016,41.7734553,-87.70648047,"(41.773455295, -87.706480471)"


In [19]:
%config SqlMagic.displaylimit = 5

##### Example 5

What is the *case number* and *community name* for all crimes in which someone got *arrested*?  

Looking at the crimes and census datasets, we see that COMMUNITY_AREA_NUMBER is a key that connects these two datasets.  

What type of join should we use?

Since having the case number and no community name for a crime is still important information, we choose a left outer join.

Before we perform this join, lets make sure we understand the values of the ARREST column.  To do so, we use a group by.

In [20]:
%sql select arrest, count(*) from crime group by arrest;

Running query in 'sqlite:///chicago.db'

ARREST,count(*)
0,370
1,163


Ok, now that we know that arrest is a binary column, we proceed with the join operation.

In [21]:
%config SqlMagic.displaylimit = None

displaylimit: Value None will be treated as 0 (no limit)

In [31]:
%%sql 

select case_number, community_area_name from crime left outer join census 
on crime.COMMUNITY_AREA_NUMBER = census.COMMUNITY_AREA_NUMBER where arrest = 1; 

Running query in 'sqlite:///chicago.db'

CASE_NUMBER,COMMUNITY_AREA_NAME
HL793243,Grand Boulevard
HX552797,West Lawn
HR271603,Calumet Heights
HY234744,Near North Side
JB151690,McKinley Park
G256545,None
HM324025,Near North Side
G662598,None
HR261581,Lower West Side
G040244,None


In [22]:
%%sql 

select count(*) from crime left outer join census on crime.COMMUNITY_AREA_NUMBER = census.COMMUNITY_AREA_NUMBER 
where arrest = 1; 

Running query in 'sqlite:///chicago.db'

count(*)
163


$\Box$

##### Example 6

What schools are in a community with per capita income greater than $50000?

Since having the school name without the community name or the community name without the school name is of no use to us, we choose an inner join.

In [23]:
%%sql 

select S.name_of_school as 'school', C.community_area_name as 'community', C.per_capita_income from schools S 
inner join census C on S.COMMUNITY_AREA_NUMBER = C.COMMUNITY_AREA_NUMBER where C.per_capita_income > 50000;

Running query in 'sqlite:///chicago.db'

school,community,PER_CAPITA_INCOME
Albert G Lane Technical High School,North Center,57123
Alcott High School for the Humanities,North Center,57123
Alexander Graham Bell Elementary School,North Center,57123
DeVry University Advantage Academy High School,North Center,57123
Friedrich Ludwig Jahn Elementary School,North Center,57123
John C Coonley Elementary School,North Center,57123
John J Audubon Elementary School,North Center,57123
Alexander Hamilton Elementary School,Lake View,60058
Augustus H Burley Elementary School,Lake View,60058
Hawthorne Elementary Scholastic Academy,Lake View,60058


$\Box$

##### Example 7

What kinds of crimes took place at a school?  What communities did this happen in?

Looking at the crimes dataset, we see that there is a location description column.  I wonder if any of the entries in this column say something about a school? 

In [25]:
%sql select LOCATION_DESCRIPTION, count(*) from crime group by LOCATION_DESCRIPTION;

Running query in 'sqlite:///chicago.db'

LOCATION_DESCRIPTION,count(*)
AIRPORT/AIRCRAFT,3
ALLEY,11
APARTMENT,59
ATHLETIC CLUB,1
BANK,5
BAR OR TAVERN,4
CHA APARTMENT,2
CHA HALLWAY/STAIRWELL/ELEVATOR,1
CHA PARKING LOT/GROUNDS,3
CHURCH/SYNAGOGUE/PLACE OF WORSHIP,5


Aha!  There is some entries having something to do with a school.

In [24]:
%%sql 

select primary_type, location_description as 'location', community_area_name from crime left outer join 
census on crime.COMMUNITY_AREA_NUMBER = census.COMMUNITY_AREA_NUMBER where location_description like '%SCHOOL%';

Running query in 'sqlite:///chicago.db'

PRIMARY_TYPE,location,COMMUNITY_AREA_NAME
BATTERY,"SCHOOL, PUBLIC, GROUNDS",South Shore
BATTERY,"SCHOOL, PUBLIC, BUILDING",Lincoln Square
BATTERY,"SCHOOL, PUBLIC, BUILDING",Douglas
BATTERY,"SCHOOL, PUBLIC, BUILDING",Austin
BATTERY,"SCHOOL, PUBLIC, GROUNDS",Ashburn
CRIMINAL DAMAGE,"SCHOOL, PUBLIC, GROUNDS",Austin
NARCOTICS,"SCHOOL, PUBLIC, GROUNDS",Rogers Park
NARCOTICS,"SCHOOL, PUBLIC, BUILDING",Brighton Park
ASSAULT,"SCHOOL, PUBLIC, GROUNDS",East Garfield Park
CRIMINAL TRESPASS,"SCHOOL, PUBLIC, GROUNDS",Ashburn


$\Box$

##### Exercise 1

Answer the following question using a SQL query.

What are the top 5 crimes that occur in a community area that has an average percapita income less than $20000?

##### Exercise 2

Answer the following question using a SQL query.

How many crimes took place in a store in a community area that has a per capita income less than $40000?

##### Exercise 3

Answer the following question using a SQL query.

What schools are in a community area with a hardship index greater than 20.0?